In [8]:
import random
import spacy
from spacy.training.example import Example
from tqdm import tqdm
from spacy import displacy

In [3]:

TRAIN_DATA = [
("Few people attended the meeting.", {
 'heads': [1,2,2,4,2,2],
 'deps': ['qauntity', 'nsubj','ROOT','det','dobj','punct']
 }),

 ("Many students completed their assignments.", {
 'heads': [1,2,2,4,2,2],
 'deps': ['qauntity', 'nsubj', 'ROOT', 'poss', 'dobj', 'punct']
 }) ,
 ("Numerous opportunities await you.", {
 'heads': [1,2,2,2,2],
 'deps': ['qauntity', 'nsubj','ROOT','dobj','punct']
 }),
("The whole team celebrated the victory.", {
 'heads': [1,2,3,3,5,3,3],
 'deps': ['det', 'quantity','nsubj','Root','det','dobj','punct']
 }),
 ("The marathon covered half the city.", {
 'heads': [1,2,2,5,5,2,2],
 'deps': ['det','nsubj','Root','quantity','det','dobj','punct']
 }),
 
 ("The concert thrilled all attendees.", {
 'heads': [1, 2, 2, 4, 2, 2],
 'deps': ['det', 'nsubj', 'ROOT', 'quantity', 'dobj', 'punct']
 }),
 ("The book interested numerous readers.", {
 'heads': [1, 2, 2, 4, 2, 2],
 'deps': ['det', 'nsubj', 'ROOT', 'quantity', 'dobj', 'punct']
 }),
  ("The store stocks some popular products.", {
 'heads': [1, 2, 2, 5,5, 2, 2],
 'deps': ['det', 'nsubj', 'ROOT', 'quantity','amod', 'dobj', 'punct']
 }),
("There were enough resources for the project.", {
 'heads': [1, 1, 3, 1,3, 6, 4,1],
 'deps': ['expel', 'ROOT', 'quantity', 'attr','prep', 'det','pobj', 'punct']
 })

]




In [4]:
# Create a blank English spaCy model.
nlp = spacy.blank("en")

# Add the POS tagger component to the pipeline.
parser = nlp.add_pipe("parser")

deps=['nsubj', 'ROOT', 'dobj', 'cc', 'conj', 'punct', 'compound', 'nmod', 'poss','prep','attr','pobj','det','quantity']
#taggs=['ROOT', 'Qauntity']
for dep in deps:
    parser.add_label(dep)

In [5]:
epochs = 50

# Disable other components to only train the POS tagger.
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'parser']

with nlp.disable_pipes(*other_pipes):
    # Initialize the optimizer for training.
    optimizer = nlp.begin_training()

    for i in tqdm(range(epochs)):
        # Shuffle the training data for randomness.
        random.shuffle(TRAIN_DATA)
        for text, annotation in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            nlp.update([example], drop=0.0, losses={})


100%|██████████| 50/50 [00:18<00:00,  2.76it/s]


In [7]:




doc = nlp("The competition featured numerous talented participants.")
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_)
displacy.render(doc, style="dep",jupyter=True)

The det competition 
competition nsubj featured 
featured ROOT featured 
numerous quantity participants 
talented poss participants 
participants dobj featured 
. punct featured 


In [6]:

nlp.to_disk('parser_nlp')

In [7]:
nlp_parser = spacy.load("parser_nlp")

In [13]:
# Test the trained POS tagger on a sample text.
doc = nlp_parser("The teacher praised all students for their efforts.")
displacy.render(doc, style="dep",jupyter=True)

In [9]:
displacy.render(nlp("Few people attended the meeting."), style="dep",jupyter=True)